## sklearn 内feature_select 的一些函数

In [16]:
from sklearn import feature_selection 
from  sklearn.datasets import load_iris
import numpy as np

In [18]:
x,y=load_iris(True)

### 1. 移除低方差特征
feature_selection.VarianceThreshold ,默认情况下移除方差为0的特征，一般应用于0,1特征
$$Var[X]=p(1-p)$$

我们要移除那些超过80%的数据都为1或0的特征：：
我们可以使用阈值 .8 * (1 - .8)

In [12]:
x = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]])
x

array([[0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 1],
       [0, 1, 0],
       [0, 1, 1]])

In [13]:
variance=feature_selection.VarianceThreshold(threshold=0.8*(1-0.8))

In [15]:
variance.fit_transform(x)

array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])

### 2. 单变量特征选择

* feature_selection.SelectKBest
* feature_selection.SelectPercentile  #选择得分较高的百分比
* feature_selection.GenericUnivariateSelect 
* 可以设置不同的策略来进行单变量特征选择。同时不同的选择策略也能够使用超参数寻优，从而让我们找到最佳的单变量特征选择策略。
    

##### SelectKBest 

feature_selection.SelectKBest(score_func=<function f_classif at 0x000001D3B92FB840>, k=10)

SelectKBest(chi2, k=2).fit_transform(X, y)

In [60]:
#top K
kbest=feature_selection.SelectKBest(feature_selection.chi2,k=3)

In [61]:
kbest.fit_transform(x,y).shape

(150, 3)

##### 查找百分数

In [32]:
per=feature_selection.SelectPercentile(percentile=70)

In [34]:
per.fit_transform(x,y).shape

(150, 3)

#### 2.2 统计分析的结果

对每个特征使用通用的单变量统计测试：  
* feature_selection.SelectFdr #伪发现率(false discovery rate
* feature_selection.SelectFpr  # 假正率(false positive rate)
* feature_selection.SelectFwe   #族系误差率 

feature_selection.chi2

Compute chi-squared stats between each non-negative feature and class.

* This score can be used to select the n_features features with the highest values for the test chi-squared statistic from X, which must
contain only non-negative features such as booleans or frequencies (e.g., term counts in document classification), relative to thec 
lasses.

* Recall that the chi-square test measures dependence between stochastic variables, so using this function "weeds out" the features that are the most likely to be independent of class and therefore irrelevant for classification.

这些对象将得分函数作为输入，返回单变量的得分和 p 值 （或者仅仅是 SelectKBest 和 SelectPercentile 的分数）:

* 对于回归: f_regression , mutual_info_regression
* 对于分类: chi2 , f_classif , mutual_info_classif

In [35]:
feature_selection.chi2

### 3. 稀疏数据的特征选择
    如果你使用sparse数据（比如：数据表示使用sparse matrics），如果不对它作dense转换，那么只有chi2 适合处理这样的数据
    注意：如果在分类问题上使用回归的scoring函数，你将得到无用的结果

总结：

* f_classif: 在label/feature之间的方差分析(Analysis of Variance:ANOVA) 的F值，用于分类.
* chi2: 非负feature的卡方检验, 用于分类.
* f_regression: 在label/feature之间的F值，用于回归.
* SelectKBest: 得到k个最高分的feature.
* SelectFpr: 基于RPR（false positive rate）检验。
* SelectFdr: 使用Benjamini-Hochberg过程。选择p值（false discovery rate）。
* SelectFwe:
* GenericUnivariateSelect: 可配置化的单变量特征选择器.

### 3. 递归特征消除(Recursive feature elimination)

feature_selection.RFE

feature_selection.RFECV #

### 4.SelectFromModel

feature_selection.SelectFromModel(estimator, threshold=None, prefit=False)

* threshold ： 阈值，string, float, optional default None 可以使用：median 或者 mean 或者 1.25 * mean 这种格式。
* 如果使用参数惩罚设置为L1，则使用的阈值为1e-5，否则默认使用用mean
*  prefit ：布尔，默认为False，是否为训练完的模型，（注意不能是cv，GridSearchCV或者clone the estimator得到的），如果是False的话则先fit，再transform。



##### 4.1 基于L1的特征选择(L1-based feature selection)
使用L1正则化的线性模型会得到稀疏解，当目标是降低维度的时候，可以使用sklearn中的给予L1正则化的线性模型，比如LinearSVC，逻辑回归，或者Lasso。

但是要注意的是：在 SVM 和逻辑回归中，参数 C 是用来控制稀疏性的：小的 C 会导致少的特征被选择。使用 Lasso，alpha 的值越大，越少的特征会被选择。

In [13]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
iris = load_iris()
X, y = iris.data, iris.target
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape


F:\app\anaconda\lib\site-packages\sklearn\utils\__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


(150, 3)

##### 4.2 随机稀疏模型(Randomized sparse models)

##### 4.3 基于树的特征选择(Tree-based feature selection)
Tree类的算法包括决策树，随机森林等会在训练后，得出不同特征的重要程度，我们也可以利用这一重要属性对特征进行选择。

##### 4.4特征选择融入pipeline(Feature selection as part of a pipeline)

    clf = Pipeline([
      ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
      ('classification', RandomForestClassifier())
    ])
    clf.fit(X, y)

### 函数方法

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
CountVectorizer?